In [26]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [77]:
options = Options()
# options.add_argument("--headless")
driver = webdriver.Chrome(options=options)
driver.get("https://www.hy-vee.com/aisles-online/browse/fresh")
time.sleep(3)  # Wait for JS to load

seen = set()
data = []
categories = ['bakery', 'fresh', 'pantry', 'beverages', 'frozen']

WAIT = 2
MAX_SCROLLS = 15
SCROLL_PAUSE = 1.5
MAX_PAGES = 100
BASE_URL = "https://www.hy-vee.com/aisles-online/browse/fresh?page={}"

In [78]:
def scroll_slowly(driver, pause_time=1.0, scroll_step=400, max_scrolls=15):
    prev_count = 0

    for i in range(max_scrolls):
        driver.execute_script(f"window.scrollBy(0, {scroll_step});")
        time.sleep(pause_time)

        # Count visible product cards after each scroll
        cards = driver.find_elements(By.CSS_SELECTOR, "[data-testid='product-card']")
        print(f"🛒 Scroll {i+1}: Found {len(cards)} cards")
        
        prev_count = len(cards)


In [79]:
# Grab all product cards
cards = driver.find_elements(By.CSS_SELECTOR, "[data-testid='product-card']")
print(f"🛒 Found {len(cards)} product cards")

data = []
seen = set()

🛒 Found 12 product cards


In [80]:
BASE_URL = "https://www.hy-vee.com/aisles-online/browse/{}"
for category in categories:
    category_url = BASE_URL.format(category) + "?page={}"
    for page in range(1, MAX_PAGES + 1):
        print(f"\n📄 Scraping page {page}")
        url = category_url.format(page)
        driver.get(url)
        time.sleep(WAIT)
        scroll_slowly(driver)

        cards = driver.find_elements(By.CSS_SELECTOR, "[data-testid='product-card']")
        print(f"🔎 Found {len(cards)} product cards")
    
        new_items = 0
        for card in cards:
            try:
                name = card.find_element(By.CSS_SELECTOR, "div[class*='ProductDescription']").text.strip()
                price = card.find_element(By.CSS_SELECTOR, "div[class*='AmountText']").text.strip()
                key = (name, price)
                if key not in seen:
                    seen.add(key)
                    data.append({"name": name, "price": price})
                    new_items += 1
            except:
                continue
    
        print(f"🆕 {new_items} new items")
    
        if new_items == 0:
            print("✅ No new items found — likely looped back to page 1. Stopping.")
            break


📄 Scraping page 1
🛒 Scroll 1: Found 15 cards
🛒 Scroll 2: Found 20 cards
🛒 Scroll 3: Found 25 cards
🛒 Scroll 4: Found 30 cards
🛒 Scroll 5: Found 35 cards
🛒 Scroll 6: Found 35 cards
🛒 Scroll 7: Found 45 cards
🛒 Scroll 8: Found 45 cards
🛒 Scroll 9: Found 50 cards
🛒 Scroll 10: Found 55 cards
🛒 Scroll 11: Found 60 cards
🛒 Scroll 12: Found 60 cards
🛒 Scroll 13: Found 60 cards
🛒 Scroll 14: Found 60 cards
🛒 Scroll 15: Found 60 cards
🔎 Found 60 product cards
🆕 60 new items

📄 Scraping page 2
🛒 Scroll 1: Found 15 cards
🛒 Scroll 2: Found 20 cards
🛒 Scroll 3: Found 25 cards
🛒 Scroll 4: Found 30 cards
🛒 Scroll 5: Found 35 cards
🛒 Scroll 6: Found 40 cards
🛒 Scroll 7: Found 40 cards
🛒 Scroll 8: Found 50 cards
🛒 Scroll 9: Found 50 cards
🛒 Scroll 10: Found 55 cards
🛒 Scroll 11: Found 60 cards
🛒 Scroll 12: Found 60 cards
🛒 Scroll 13: Found 60 cards
🛒 Scroll 14: Found 60 cards
🛒 Scroll 15: Found 60 cards
🔎 Found 60 product cards
🆕 60 new items

📄 Scraping page 3
🛒 Scroll 1: Found 15 cards
🛒 Scroll 2: Fo

In [81]:
df = pd.DataFrame(data)
df.to_csv("hyvee_scraped_products.csv", index=False)
print(f"✅ Scraped and saved {len(df)} products to hyvee_scraped_products.csv")

✅ Scraped and saved 3500 products to hyvee_scraped_products.csv
